In [8]:
!pip3 install dash




     |████████████████████████████████| 10.4 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 4.2 MB/s ta 0:00:01
     |████████████████████████████████| 233 kB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.6 MB/s eta 0:00:01
  Using cached idna-3.4-py3-none-any.whl (61 kB)
     |████████████████████████████████| 118 kB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 124 kB 9.1 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px



In [38]:
df = pd.read_json('frames.json')


In [39]:
df = df['objects'].apply(lambda x : [i['name'] for i in x]).explode()
df = pd.DataFrame({ "second" : df.index, "object" : df.values})

In [40]:
df

,second,object
0,0,Building
1,0,Building
2,0,Building
3,0,Building
4,1,Window
...,...,...
4323,1045,Person
4324,1046,Toy
4325,1047,Toy
4326,1047,Toy


In [78]:
#MAKE THIS A FUNCTION


# Group by second and object name, then pivot to get a column per object with counts
pivot_df = df.groupby(['second', 'object']).size().unstack(fill_value=0).reset_index()

# Define time interval
interval = 20
pivot_df['time_interval'] = (pivot_df['second'] // interval) * interval

# Group by the time interval and sum the counts
agg_df = pivot_df.groupby('time_interval').sum().drop(columns=['second'])

# Identify the top n objects across all intervals
n = 20
top_objects = agg_df.sum().sort_values(ascending=False).head(n).index.tolist()

# Filter the aggregate dataframe to only include the top n objects
agg_df = agg_df[top_objects]

# Plot
fig = px.bar(agg_df, x=agg_df.index, y=top_objects, title="Top Object Appearances Over Time Intervals", labels={"value": "Number of Appearances", "time_interval": "Time Interval"}, height=400)
fig.update_layout(barmode='stack', xaxis_title=f"Time (in intervals of {interval} seconds)", yaxis_title='Number of Objects')
fig.show()





In [79]:
#maybe this too
# Filter columns for the objects you want to compare
selected_objects = ["Hat", "Wheel"]
filtered_df = pivot_df[["second"] + selected_objects]

# Melt dataframe to be suitable for line plot with plotly
melted_df = filtered_df.melt(id_vars=["second"], value_vars=selected_objects, var_name="Object", value_name="Frequency")

# Create line plot
fig = px.line(melted_df, x="second", y="Frequency", color="Object", title="Comparison of Object Appearances Over Time")
fig.update_layout(xaxis_title='Second', yaxis_title='Number of Appearances')
fig.show()

In [45]:
df_freq = df['object'].value_counts().reset_index()

df_freq


,object,count
0,Person,1396
1,Building,1040
2,Car,513
3,Clothing,432
4,Pants,69
...,...,...
70,Grooming trimmer,1
71,Bench,1
72,Ship,1
73,Overall,1


In [76]:

# Define a function to get objects in a specified range
def get_objects_in_range(df, start=1, end=10):
    # Sort the dataframe
    sorted_df = df.sort_values(by='count', ascending=False)
    print(sorted_df)
    
    # Get the specified range. Note that Python's list slicing is 0-based, 
    # so we subtract 1 from start and end to get the correct indices.
    return sorted_df.iloc[start-1:end]

# Get objects ranked 11 to 20
objects_11_to_20 = get_objects_in_range(df_freq, start=1,end=10)

# Plot
fig = px.bar(objects_11_to_20, x='count', y='object', orientation='h', color='object', title='Frequency Distribution of Object Appearances', labels={'count': 'Number of Appearances', 'object': 'Object Type'})
fig.update_layout(xaxis_title='Number of Appearances', yaxis_title='Object Type')
fig.show()

          object  count
0         Person   1396
1       Building   1040
2            Car    513
3       Clothing    432
4          Pants     69
..           ...    ...
62         Scarf      1
61          Lamp      1
60     Miniskirt      1
59  Window blind      1
74          Food      1

[75 rows x 2 columns]
